In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/smile')

In [3]:
import random

import numpy as np
import torch
import torch.nn as nn
# from torchsummary import summary
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils import data
from torchvision import models
from tqdm import tqdm
from PIL import Image
import glob
import pickle

RANDOM_SEED = 41
np.random.seed(RANDOM_SEED)

random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [0]:
def get_model_regression(out_dim=1024, pretrained=False):
    model = models.resnet18(pretrained=pretrained)
    nb_features = model.fc.in_features
    model.fc = nn.Sequential(nn.Linear(nb_features, out_dim), nn.Sigmoid())
    #model.fc_1 = nn.Sigmoid()
    if target_channels == 1:
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    return model

In [0]:
def custom_loss(anchor, positive_negative, label):
  cosim = nn.CosineSimilarity()
  cosim_output = cosim(anchor, positive_negative)
  bceloss = torch.nn.BCELoss()
  loss = bceloss(1 - cosim_output, label)
  return loss

In [0]:
def create_array(folder):
  image_map = {}
  for filename in glob.glob(f'{folder}/*'):
#     image = Image.open(filename).convert('RGB')
#     image = image.resize((target_width, target_height))
#     image = np.array(image) / 255.
    feature_array = np.load(filename)
    image_map[filename] = feature_array
  with open('image_mapping_dictionary.pickle', 'wb') as handle:
    pickle.dump(image_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

  

In [0]:
def load_checkpoint(model, optimizer, filename='checkpoint.pth.tar', checkpoint_epoch=0):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = checkpoint_epoch
    if os.path.isfile(filename):
        print(f"=> loading checkpoint '{filename}'")
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print(f"=> loaded checkpoint '{filename}' (epoch {checkpoint['epoch']})")
    else:
        print(f"=> no checkpoint found at '{filename}'")
        start_epoch = 0
    return model, optimizer, start_epoch

In [0]:
class Smile(data.Dataset):

    def __init__(self, X, pickle_file):
        self.X = X
        with open(f'{pickle_file}', 'rb') as handle:
          self.image_map = pickle.load(handle)
          
    def __getitem__(self, index):
        doublets = np.zeros((2, target_width, target_height, target_channels))
        X = self.X[index]
        doublets[0] = self.image_map[X[0]]
        doublets[1] = self.image_map[X[1]]
        doublets = torch.from_numpy(doublets.transpose((0, 3, 1, 2))).type(torch.FloatTensor)
        return doublets, torch.tensor(int(X[2])).type(torch.FloatTensor)

    def __len__(self):
        return len(self.X)

In [0]:
def train(weights_folder, data_folder, X_train, X_val, params, pretrained = False, checkpoint_epoch=0):
    epochs = params['epochs']
    batch_size = params['batch_size']
    lr = params['learning_rate']
    weight_decay = params['weight_decay']
    #alpha = params['alpha']
    model_name = params["model_name"]


    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True

    train_dataset = Smile(X_train, data_folder)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,
                              worker_init_fn=np.random.seed(42))
    val_dataset = Smile(X_val, data_folder)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0,
                              worker_init_fn=np.random.seed(42))
    
    model = get_model_regression(pretrained=pretrained)
    
    if torch.cuda.is_available():
        model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)
    
    
    weight_filename = f'{weights_folder}/{model_name}_{checkpoint_epoch}.pt'
    model, optimizer, start_epoch = load_checkpoint(model, optimizer, filename=weight_filename,
                                                    checkpoint_epoch=checkpoint_epoch)
    if torch.cuda.is_available():
        model = model.cuda()
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                if torch.cuda.is_available():
                    state[k] = v.cuda()
    
    
    valid_loss_min = np.Inf
    train_size = len(train_loader)
    validation_size = len(val_loader)

    for epoch in range(start_epoch+1, epochs+1):
        print(epoch)
        train_loss = 0.
        valid_loss = 0.

        # TRAIN
        with tqdm(total=train_size, desc=f"Training Epoch {epoch}",
                  bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
            model.train()
            for inputs, label in train_loader:
                #print(inputs.shape)
                pbar.update(1)
                if torch.cuda.is_available():
                    inputs = inputs.cuda()
                    label = label.cuda()
                optimizer.zero_grad()
                
                input_0 = inputs[:, 0]
                #print(input_0.shape)
                output_0 = model(input_0)
                input_1 = inputs[:, 1]
                output_1 = model(input_1)
                loss = custom_loss(output_0, output_1, label)
                loss.backward()
                optimizer.step()
                train_loss += loss.item() * inputs.size(0)

        # VALIDATION
        with tqdm(total=validation_size, desc=f"Validating",
                  bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
            model.eval()
            for inputs,label in val_loader:
                pbar.update(1)
                if torch.cuda.is_available():
                    inputs = inputs.cuda()
                    label = label.cuda()
                optimizer.zero_grad()           
                input_0 = inputs[:, 0]
                #print(input_0.shape)
                output_0 = model(input_0)
                input_1 = inputs[:, 1]
                output_1 = model(input_1)
                loss = custom_loss(output_0, output_1, label)
                valid_loss += loss.item() * inputs.size(0)

            train_loss = train_loss / len(train_loader.dataset)
            valid_loss = valid_loss / len(val_loader.dataset)
            print(f'Epoch: {epoch} \tTraining loss: {train_loss} \tValidation loss: {valid_loss}')

        scheduler.step(valid_loss)

        # MODEL SAVING
        if valid_loss <= valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min} --> {valid_loss}).  Saving model ...')
            state = {'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
            torch.save(state, f'{weights_folder}/{model_name}_{epoch}.pt')
            valid_loss_min = valid_loss

    print(f"Best validation Loss: {valid_loss_min}")

In [12]:
if __name__ == '__main__':
    doublet_data = np.load('doublets.npy')
    validation_size = 0.2
    target_width, target_height, target_channels = 224, 224, 3
    X_train, X_val = train_test_split(doublet_data, test_size=validation_size, random_state=RANDOM_SEED)
    create_array('single_images')
    train('weights', 'image_mapping_dictionary.pickle', X_train, X_val, params={
        "epochs": 300,
        "batch_size": 8,
        "learning_rate": 5e-5,
        "weight_decay": 0,
        "model_name": "best_model"
        #"alpha": 0.3
    }, pretrained = True, checkpoint_epoch=0)

Training Epoch 1:   0%|           [ time left: ? ]

=> no checkpoint found at 'weights/best_model_0.pt'
1


Training Epoch 1: 100%|██████████ [ time left: 00:00 ]
Validating: 100%|██████████ [ time left: 00:00 ]


Epoch: 1 	Training loss: 0.8874515952459022 	Validation loss: 0.7145727556143234
Validation loss decreased (inf --> 0.7145727556143234).  Saving model ...


Training Epoch 2:   0%|           [ time left: ? ]

...

Epoch: 26 	Training loss: 0.042426922245284736 	Validation loss: 0.6339633737357139
Epoch    25: reducing learning rate of group 0 to 1.2500e-05.
27


Training Epoch 27: 100%|██████████ [ time left: 00:00 ]
Validating: 100%|██████████ [ time left: 00:00 ]
Training Epoch 28:   0%|           [ time left: ? ]

Epoch: 27 	Training loss: 0.035071018608564986 	Validation loss: 0.6344043984689486
28


Training Epoch 28: 100%|██████████ [ time left: 00:00 ]
Validating:  92%|█████████▏ [ time left: 00:01 ]


KeyboardInterrupt: ignored